In [4]:
import pandas as pd
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [5]:
df=pd.read_csv(r"C:/Users/azizd/Desktop/Miull/Machine Learning/machine_learning-220803-231749/machine_learning/datasets/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df.Outcome.value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [10]:
y= df.Outcome
X = df.drop("Outcome", axis=1)

In [11]:
X_scaled = StandardScaler().fit_transform(X)

X=pd.DataFrame(X_scaled, columns=X.columns)
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


In [12]:
knn_model= KNeighborsClassifier().fit(X, y)

In [14]:
random_user=X.sample(1, random_state=45)

knn_model.predict(random_user)

array([1], dtype=int64)

In [15]:
y_pred= knn_model.predict(X)

y_pred_proba= knn_model.predict_proba(X)[:,1]

print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



In [17]:
print(roc_auc_score(y, y_pred_proba))

0.9017686567164179


In [28]:
cv_results=cross_validate(knn_model,X,y, cv=5, scoring=["accuracy","f1", "roc_auc"])

cv_results

{'fit_time': array([0.00795984, 0.0058012 , 0.        , 0.        , 0.00307584]),
 'score_time': array([0.05744171, 0.03420806, 0.0506475 , 0.04547811, 0.03587437]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

In [29]:
print(cv_results["test_accuracy"].mean())
print(cv_results["test_f1"].mean())
print(cv_results["test_roc_auc"].mean())

0.733112638994992
0.5905780011534191
0.7805279524807827


In [27]:
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [31]:
knn_params= {"n_neighbors": range(2,50)}

knn_gs_best=GridSearchCV(knn_model,
						 knn_params,
						 cv=5, 
						 n_jobs=-1,
						verbose=1).fit(X,y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [32]:
knn_gs_best.best_params_

{'n_neighbors': 17}

In [33]:
knn_final=knn_model.set_params(**knn_gs_best.best_params_).fit(X,y)

cv_results=cross_validate(knn_final,X,y, cv=5, scoring=["accuracy","f1", "roc_auc"])

print(cv_results["test_accuracy"].mean())
print(cv_results["test_f1"].mean())
print(cv_results["test_roc_auc"].mean())

0.7669892199303965
0.6170909049720137
0.8127938504542278
